In [1]:
import pandas as pd
import numpy as np

In [2]:
#import and show CSV file
df = pd.read_csv('./CSV/group_data.csv')
df.head(5)

,Report Date,Group#,Group Zip,Medical Enroll Month/Day,Medical Enroll Year,Renewal Date,Term Date,Length (in months),Term Reason Code,Term Reason (numerical),...,Life Type,Vision,Vision Type,Sic Code,Waiting Period,Account Manager,Areas/Regions 1-19,EnrollmentDate,Medical Total # Deps (15-26),Status
0,3/1/2018,42772.0,92009.0,6/1/2018,2014.0,6/1/2017,5/31/2016,23,NP,1,...,NaN,NaN,NaN,8748.0,60.0,ASCHE,NaN,6/1/2014,NaN,TERMED
1,3/1/2018,42773.0,92270.0,6/1/2018,2014.0,6/1/2018,5/31/2017,35,NR,2,...,NaN,NaN,NaN,8093.0,30.0,NaN,NaN,6/1/2014,NaN,TERMED
2,3/1/2018,42774.0,90011.0,6/1/2018,2014.0,6/1/2018,NaN,52,NaN,3,...,NaN,NaN,NaN,3713.0,30.0,PLASC,NaN,6/1/2014,NaN,ACTIVE
3,3/1/2018,42775.0,91406.0,6/1/2018,2014.0,6/1/2015,12/31/2014,6,NP,1,...,NaN,NaN,NaN,1743.0,30.0,PLASC,NaN,6/1/2014,NaN,TERMED
4,3/1/2018,42777.0,91306.0,6/1/2018,2014.0,9/1/2018,8/31/2017,38,CH,4,...,NaN,NaN,NaN,8211.0,30.0,NaN,NaN,6/1/2014,NaN,TERMED


In [3]:
#clean up data to only show needed columns
clean_df = df.filter(['Group Zip', 'Length (in months)', 'Term Reason (numerical)','Medical Total Count','Sic Code', 'Status'])
                     

In [4]:
# Drop NaN
group_df = clean_df.dropna(how='any')
group_df.head(5)

,Group Zip,Length (in months),Term Reason (numerical),Medical Total Count,Sic Code,Status
0,92009.0,23,1,0,8748.0,TERMED
1,92270.0,35,2,0,8093.0,TERMED
2,90011.0,52,3,36,3713.0,ACTIVE
3,91406.0,6,1,0,1743.0,TERMED
4,91306.0,38,4,0,8211.0,TERMED


In [5]:
x = group_df.drop("Status", axis=1)
y = group_df["Status"]
print(x.shape, y.shape)

(195629, 5) (195629,)


In [6]:
print (group_df.dtypes)

Group Zip                  float64
Length (in months)           int64
Term Reason (numerical)      int64
Medical Total Count          int64
Sic Code                   float64
Status                      object
dtype: object


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

x_train, x_test, y_train, y_test = train_test_split(
    x, y, random_state=1, stratify=y)
x_scaler = StandardScaler().fit(x_train)
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)


C:\Users\eric_\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [9]:
from keras.models import Sequential
from keras.layers import Dense

# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=5))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [10]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    x_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
 - 4s - loss: 0.0181 - acc: 0.9947
Epoch 2/60
 - 3s - loss: 0.0029 - acc: 0.9996
Epoch 3/60
 - 3s - loss: 0.0025 - acc: 0.9997
Epoch 4/60
 - 3s - loss: 0.0016 - acc: 0.9998
Epoch 5/60


KeyboardInterrupt: 

In [11]:
model_loss, model_accuracy = model.evaluate(
    x_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.0007927414798700878, Accuracy: 0.999897767236444


In [12]:
encoded_predictions = model.predict_classes(x_test_scaled[:20])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

C:\Users\eric_\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:20])}")

Predicted classes: ['TERMED' 'ACTIVE' 'TERMED' 'TERMED' 'TERMED' 'TERMED' 'TERMED' 'TERMED'
 'TERMED' 'TERMED' 'TERMED' 'ACTIVE' 'TERMED' 'ACTIVE' 'TERMED' 'TERMED'
 'ACTIVE' 'TERMED' 'TERMED' 'TERMED']
Actual Labels: ['TERMED', 'ACTIVE', 'TERMED', 'TERMED', 'TERMED', 'TERMED', 'TERMED', 'TERMED', 'TERMED', 'TERMED', 'TERMED', 'ACTIVE', 'TERMED', 'ACTIVE', 'TERMED', 'TERMED', 'ACTIVE', 'TERMED', 'TERMED', 'TERMED']


In [14]:
# Save the model
model.save("group_trained.h5")